In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [7]:
print(char_set, len(char_set))
print(char_dic, len(char_dic))

['l', 'y', ' ', 'o', 's', 'm', "'", 'w', 'b', 'd', 'c', 't', 'p', 'e', '.', 'h', 'u', 'k', 'r', 'f', ',', 'n', 'i', 'g', 'a'] 25
{'l': 0, 'y': 1, ' ': 2, 'o': 3, 's': 4, 'm': 5, "'": 6, 'w': 7, 'b': 8, 'd': 9, 'c': 10, 't': 11, 'p': 12, 'e': 13, '.': 14, 'h': 15, 'u': 16, 'k': 17, 'r': 18, 'f': 19, ',': 20, 'n': 21, 'i': 22, 'g': 23, 'a': 24} 25


In [8]:
# hyper parameters
dic_size = len(char_dic)      #25개, 입력 차원
hidden_size = len(char_dic)   #25개 히든 레이어 차원
sequence_length = 10          # Any arbitrary number
learning_rate = 0.1

In [12]:
print(len(sentence), sequence_length)

180 10


In [14]:
# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
#     print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

In [18]:
print(x_one_hot[0])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0.]]


In [19]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [20]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x


net = Net(dic_size, hidden_size, 2)

In [21]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [23]:
# start training
for i in range(1000):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

l woo tant to build anseip, do 't drum up peo le to ether to bo le t wood and do gt dssigngthe  tosk  and wook, but dather teach them to long for the tnd ess imm nsity o  the thns
l go  want to butld anseip, do 't drul up per le to ether th bo lect word and do 't dssign them tosks andlaork, but dather thach them to long for the tndless immensity of thentens
m go  want to cuild anseip, don't aoum up peo le to ether te co lect word tnd don't dnsitn the  tosks andlaork, but dather teash them to long forbthemtndless immensity of therthns
m yo  want to cuild anship, don't drum up peo le to ether to co lect wood tnd don't dnsign the  tosks and aork, but dathe  thach them to long for the tnd ess immensity of the thns
p yo  want to cutld anship, don't drum dp peo le to ethe  to co lect wood and don't dssign the  tasks and dork, but aathe  thach the  to long for the snd esssimmensity of the shns
p yoo want to cuild anship, don't drum up peo le to ether to co lect word and don't dssign the  task